In [12]:
from cellpose import models, core, io, plot

#Check if colab notebook instance has GPU access
if core.use_gpu()==False:
  raise ImportError("No GPU access, change your runtime")

ImportError: No GPU access, change your runtime

In [13]:
# import napari
from pathlib import Path
# from skimage import io
import os, glob
import czifile
import numpy as np
from tqdm.auto import tqdm
# from natsorted import natsort

In [4]:

# Path to your files
files_dir = Path(r'Z:\Zeiss\Rebecca\zeisslightsheet7\20250814')

# Use string when passing to glob
files = glob.glob(str(files_dir / '*.czi'))

# Get file sizes in bytes
file_sizes = [os.path.getsize(f) for f in files]

# Convert sizes
file_sizes_mb = [s / (1024**2) for s in file_sizes]
total_size_gb = sum(file_sizes) / (1024**3)

# Print summary
for f, s in zip(files, file_sizes_mb):
    print(f"{Path(f).name}: {s:.2f} MB")

print(f"\nNumber of files: {len(files)}")
print(f"Total stacked size: {total_size_gb:.2f} GB")

20250814_gfapgfp_tp1mch_2dpf_40x_zls7_2025_08_14__20_40_18_733(1).czi: 5653.62 MB
20250814_gfapgfp_tp1mch_2dpf_40x_zls7_2025_08_14__20_40_18_733(10).czi: 5653.62 MB
20250814_gfapgfp_tp1mch_2dpf_40x_zls7_2025_08_14__20_40_18_733(100).czi: 5653.62 MB
20250814_gfapgfp_tp1mch_2dpf_40x_zls7_2025_08_14__20_40_18_733(101).czi: 5653.62 MB
20250814_gfapgfp_tp1mch_2dpf_40x_zls7_2025_08_14__20_40_18_733(102).czi: 5653.62 MB
20250814_gfapgfp_tp1mch_2dpf_40x_zls7_2025_08_14__20_40_18_733(103).czi: 5653.62 MB
20250814_gfapgfp_tp1mch_2dpf_40x_zls7_2025_08_14__20_40_18_733(104).czi: 5653.62 MB
20250814_gfapgfp_tp1mch_2dpf_40x_zls7_2025_08_14__20_40_18_733(105).czi: 5653.62 MB
20250814_gfapgfp_tp1mch_2dpf_40x_zls7_2025_08_14__20_40_18_733(106).czi: 5653.62 MB
20250814_gfapgfp_tp1mch_2dpf_40x_zls7_2025_08_14__20_40_18_733(107).czi: 5653.62 MB
20250814_gfapgfp_tp1mch_2dpf_40x_zls7_2025_08_14__20_40_18_733(108).czi: 5653.62 MB
20250814_gfapgfp_tp1mch_2dpf_40x_zls7_2025_08_14__20_40_18_733(109).czi: 5653.6

In [5]:
test_fn = Path(files_dir, '20250814_gfapgfp_tp1mch_2dpf_40x_zls7_2025_08_14__20_40_18_733(152).czi')

In [6]:
%%time

image = np.squeeze(czifile.imread(test_fn))

CPU times: total: 17.3 s
Wall time: 32.5 s


In [7]:
image.shape

(2, 402, 1920, 1920)

In [14]:

model = models.CellposeModel(gpu=False)

100%|█████████████████████████████████████████████████████████████████████████████| 1.15G/1.15G [00:22<00:00, 54.3MB/s]


In [16]:

# --- robust per-channel normalization to [0,1] ---
seg_input = image.astype(np.float32)
for c in tqdm(range(seg_input.shape[0]), desc="Creating pseudo seg input"):
    p1, p99 = np.percentile(seg_input[c], (1, 99))
    seg_input[c] = np.clip((seg_input[c] - p1) / max(p99 - p1, 1e-6), 0, 1)


Creating pseudo seg input: 100%|██████████████████████████████████████████████████████████████████████████████████████████| 2/2 [01:55<00:00, 57.89s/it]


In [17]:
# --- pseudo max-projection across channels for each Z ---
# result: (Z, Y, X)
seg_input = np.max(seg_input, axis=0)

# --- Cellpose 2D over Z ---
masks = []
for z in tqdm(range(seg_input.shape[0]), desc="Cellpose on pseudo-max Z"):
    mask, _, _, _ = model.eval(seg_input[z])
    masks.append(mask)
masks = np.stack(masks)  # (Z, Y, X)


Cellpose on pseudo-max Z:   0%|                                                                                                 | 0/402 [24:50<?, ?it/s]


KeyboardInterrupt: 

In [ ]:

# --- visualize in napari ---
viewer = napari.Viewer(title="Pseudo-max segmentation")
viewer.add_image(combo, name="combo(ZYX)")
viewer.add_labels(masks, name="masks")
print("combo:", combo.shape, "masks:", masks.shape)